In [205]:
import numpy as np
import pandas as pd


In [206]:
cim=r'E:/Python_data Science/anaytics_vidhya/coupon_item_mapping.csv'
cdemo=r'E:/Python_data Science/anaytics_vidhya/customer_demographics.csv'
camdata=r'E:/Python_data Science/anaytics_vidhya/campaign_data.csv'
train=r'E:/Python_data Science/anaytics_vidhya/train.csv'
ctd=r'E:/Python_data Science/anaytics_vidhya/customer_transaction_data.csv'
item=r'E:/Python_data Science/anaytics_vidhya/item_data.csv'

In [207]:
cim=pd.read_csv(cim)
cdemo=pd.read_csv(cdemo)
camdata=pd.read_csv(camdata)
train=pd.read_csv(train)
ctd=pd.read_csv(ctd)
item=pd.read_csv(item)

In [208]:
test_file=r'E:/Python_data Science/anaytics_vidhya/test_QyjYwdj.csv'
test=pd.read_csv(test_file)

In [209]:
test.nunique()

id             50226
campaign_id       10
coupon_id        331
customer_id     1250
dtype: int64

In [210]:
cim.head()

,coupon_id,item_id
0,105,37
1,107,75
2,494,76
3,522,77
4,518,77


In [211]:
for col in ['start_date','end_date']:
    camdata[col]=pd.to_datetime(camdata[col])

In [212]:
camdata['duration']=(camdata['end_date']-camdata['start_date']).dt.days

In [213]:
for col in ['start_date','end_date']:
    camdata[col+'_dow']=camdata[col].dt.dayofweek
    camdata[col+'_day']=camdata[col].dt.dayofyear


In [214]:
del camdata['start_date']
del camdata['end_date']

# merge train/test campign data + customer demographics


In [215]:
train_all=pd.merge(train,camdata,on='campaign_id',how='left')
test_all=pd.merge(test,camdata,on='campaign_id',how='left')

In [216]:
train_all=pd.merge(train_all,cdemo,on='customer_id',how='left')
test_all=pd.merge(test_all,cdemo,on='customer_id',how='left')

In [217]:
train_all.shape

(78369, 17)

In [218]:
test_all.shape

(50226, 16)

# coupon_item

In [219]:
item['brand_type'].value_counts()

Established    62842
Local          11224
Name: brand_type, dtype: int64

In [220]:
coupon_item=pd.merge(cim,item,on='item_id',how='left')

In [221]:
coupon_features=coupon_item['coupon_id'].value_counts().reset_index()


In [222]:
coupon_features.columns=['coupon_id','coupon_occurence_count']

In [223]:
temp=coupon_item.groupby(['coupon_id'])['item_id'].nunique().reset_index()
coupon_features=pd.merge(coupon_features,temp,on='coupon_id',how='left')
del temp
coupon_features.rename(columns={'item_id':'coupon_unique_items_count'},inplace=True)

In [224]:
temp=pd.crosstab(coupon_item['coupon_id'],coupon_item['category'])
temp.columns=[x+'_cat_count' for x in list(temp.columns)]
temp.reset_index(inplace=True)
coupon_features=pd.merge(coupon_features,temp,on='coupon_id',how='left')
del temp

In [225]:
temp=coupon_item.groupby(['coupon_id'])['category'].nunique().reset_index()
coupon_features=pd.merge(coupon_features,temp,on='coupon_id',how='left')
del temp
coupon_features.rename(columns={'category':'coupon_unique_category_count'},inplace=True)

In [226]:
coupon_item.nunique()

coupon_id      1116
item_id       36289
brand          2555
brand_type        2
category         17
dtype: int64

In [227]:
temp=pd.crosstab(coupon_item['coupon_id'],coupon_item['brand_type'])
temp.columns=[x+'_brand_count' for x in list(temp.columns)]
temp.reset_index(inplace=True)
coupon_features=pd.merge(coupon_features,temp,on='coupon_id',how='left')
del temp

In [228]:
temp=coupon_item.groupby(['coupon_id'])['brand_type'].nunique().reset_index()
coupon_features=pd.merge(coupon_features,temp,on='coupon_id',how='left')
del temp
coupon_features.rename(columns={'brand_type':'coupon_unique_brand_type_count'},inplace=True)

In [229]:
train_all=pd.merge(train_all,coupon_features,on='coupon_id',how='left')
test_all=pd.merge(test_all,coupon_features,on='coupon_id',how='left')

In [230]:
train_all.shape

(78369, 40)

In [231]:
test_all.shape

(50226, 39)

# customer_item

In [232]:
cust_item=pd.merge(ctd,item,on='item_id',how='left')


In [233]:
cust_item

,date,customer_id,item_id,quantity,selling_price,other_discount,coupon_discount,brand,brand_type,category
0,2012-01-02,1501,26830,1,35.26,-10.69,0.0,56,Local,Natural Products
1,2012-01-02,1501,54253,1,53.43,-13.89,0.0,56,Local,Natural Products
2,2012-01-02,1501,31962,1,106.50,-14.25,0.0,524,Established,Grocery
3,2012-01-02,1501,33647,1,67.32,0.00,0.0,1134,Established,Grocery
4,2012-01-02,1501,48199,1,71.24,-28.14,0.0,524,Established,Grocery
5,2012-01-02,1501,57397,1,71.24,-28.14,0.0,524,Established,Grocery
6,2012-01-02,857,12424,1,106.50,-14.25,0.0,971,Established,Grocery
7,2012-01-02,857,14930,1,110.07,0.00,0.0,3235,Established,Meat
8,2012-01-02,857,16657,1,89.05,-35.26,0.0,2011,Established,Packaged Meat
9,2012-01-02,67,10537,3,32.06,0.00,0.0,487,Established,Grocery


In [234]:
cust_features=cust_item['customer_id'].value_counts().reset_index()
cust_features.columns=['customer_id','cust_occurence_count']

In [235]:
temp=cust_item.groupby(['customer_id'])['item_id'].nunique().reset_index()
cust_features=pd.merge(cust_features,temp,on='customer_id',how='left')
del temp
cust_features.rename(columns={'item_id':'cust_unique_items_count'},inplace=True)

In [236]:
cust_item.nunique()

date                 549
customer_id         1582
item_id            74063
quantity            9252
selling_price       4923
other_discount      1418
coupon_discount      232
brand               5528
brand_type             2
category              19
dtype: int64

In [237]:
temp=pd.crosstab(cust_item['customer_id'],cust_item['category'])
temp.columns=[x+'_cust_cat_count' for x in list(temp.columns)]
temp.reset_index(inplace=True)
cust_features=pd.merge(cust_features,temp,on='customer_id',how='left')
del temp


In [238]:
temp=cust_item.groupby(['customer_id'])['category'].nunique().reset_index()
cust_features=pd.merge(cust_features,temp,on='customer_id',how='left')
del temp
cust_features.rename(columns={'category':'cust_unique_category_count'},inplace=True)

In [239]:
temp=pd.crosstab(cust_item['customer_id'],cust_item['brand_type'])
temp.columns=[x+'_cust_brand_count' for x in list(temp.columns)]
temp.reset_index(inplace=True)
cust_features=pd.merge(cust_features,temp,on='customer_id',how='left')
del temp


In [240]:
temp=cust_item.groupby(['customer_id'])['brand_type'].nunique().reset_index()
cust_features=pd.merge(cust_features,temp,on='customer_id',how='left')
del temp
cust_features.rename(columns={'brand_type':'cust_unique_brand_type_count'},inplace=True)

In [241]:
temp=cust_item.groupby(['customer_id'])['coupon_discount'].describe()
temp.columns=[x+'_coupon_discount' for x in list(temp.columns)]
temp.reset_index(inplace=True)
cust_features=pd.merge(cust_features,temp,on='customer_id',how='left')

del temp

In [242]:
temp=cust_item.groupby(['customer_id'])['coupon_discount'].nunique().reset_index()
cust_features=pd.merge(cust_features,temp,on='customer_id',how='left')
del temp
cust_features.rename(columns={'coupon_discount':'cust_unique_coupon_discount_count'},inplace=True)

In [243]:
temp=cust_item.groupby(['customer_id'])['other_discount'].describe()
temp.columns=[x+'_other_discount' for x in list(temp.columns)]
temp.reset_index(inplace=True)
cust_features=pd.merge(cust_features,temp,on='customer_id',how='left')
del temp

In [244]:
temp=cust_item.groupby(['customer_id'])['other_discount'].nunique().reset_index()
cust_features=pd.merge(cust_features,temp,on='customer_id',how='left')
del temp
cust_features.rename(columns={'other_discount':'cust_unique_other_discount_count'},inplace=True)

In [245]:
temp=cust_item.groupby(['customer_id'])['selling_price'].describe()
temp.columns=[x+'_selling_price' for x in list(temp.columns)]
temp.reset_index(inplace=True)
cust_features=pd.merge(cust_features,temp,on='customer_id',how='left')
del temp

In [246]:
temp=cust_item.groupby(['customer_id'])['selling_price'].nunique().reset_index()
cust_features=pd.merge(cust_features,temp,on='customer_id',how='left')
del temp
cust_features.rename(columns={'selling_price':'cust_unique_selling_price_count'},inplace=True)

In [247]:
temp=cust_item.groupby(['customer_id'])['quantity'].describe()
temp.columns=[x+'_quantity' for x in list(temp.columns)]
temp.reset_index(inplace=True)
cust_features=pd.merge(cust_features,temp,on='customer_id',how='left')
del temp

In [248]:
temp=cust_item.groupby(['customer_id'])['quantity'].nunique().reset_index()
cust_features=pd.merge(cust_features,temp,on='customer_id',how='left')
del temp
cust_features.rename(columns={'quantity':'cust_unique_quantity_count'},inplace=True)

In [249]:
cust_item_sub=cust_item[['customer_id','date']]

cust_item_sub['date']=pd.to_datetime(cust_item_sub['date'])

cust_item_sub.sort_values(['customer_id','date'],inplace=True)

temp=cust_item_sub.drop_duplicates()

temp['lag_date']=temp.groupby('customer_id')['date'].shift()

temp.dropna(inplace=True)

temp['diff']=(temp['date']-temp['lag_date']).dt.days

temp1=temp.groupby('customer_id')['diff'].mean().reset_index()

cust_features=pd.merge(cust_features,temp1,on='customer_id',how='left')
del temp
cust_features.rename(columns={'date':'cust_unique_date_count'},inplace=True)


C:\Users\deepak kumar\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\deepak kumar\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
C:\Users\deepak kumar\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.p

In [ ]:
# (pd.to_datetime('today')-cust_item_sub.groupby('customer_id')['date'].nth(0)).dt.days
# (pd.to_datetime('today')-cust_item_sub.groupby('customer_id')['date'].nth(-1)).dt.days

In [250]:
train_all=pd.merge(train_all,cust_features,on='customer_id',how='left')
test_all=pd.merge(test_all,cust_features,on='customer_id',how='left')

In [251]:
train_all.shape

(78369, 102)

In [252]:
test_all.shape

(50226, 101)

In [253]:
train_all['redemption_status'].value_counts()

0    77640
1      729
Name: redemption_status, dtype: int64

# data  preparation part

In [254]:
test_all['redemption_status']=np.nan
train_all['data']='train'
test_all['data']='test'
test_all=test_all[train_all.columns]
amex_all=pd.concat([train_all,test_all],axis=0)

In [255]:
amex_all['campaign_type_X']=np.where(amex_all['campaign_type']=='X',1,0)
amex_all['campaign_type_Y']=np.where(amex_all['campaign_type']=='Y',1,0)
amex_all.drop('campaign_type',axis=1,inplace=True)


In [256]:
cat_cols=amex_all.select_dtypes('object').columns

In [257]:
cat_cols

Index(['age_range', 'marital_status', 'family_size', 'no_of_children', 'data'], dtype='object')

In [258]:
amex_all['marital_status_married']=np.where(amex_all['marital_status']=='Married',1,0)
amex_all['marital_status_single']=np.where(amex_all['marital_status']=='Single',1,0)
amex_all.drop('marital_status',axis=1,inplace=True)

In [259]:
amex_all['family_size_1']=np.where(amex_all['family_size']=='1',1,0)
amex_all['family_size_2']=np.where(amex_all['family_size']=='2',1,0)
amex_all['family_size_3']=np.where(amex_all['family_size']=='3',1,0)
amex_all['family_size_4']=np.where(amex_all['family_size']=='4',1,0)
amex_all['family_size_5']=np.where(amex_all['family_size']=='5+',1,0)
amex_all.drop('family_size',axis=1,inplace=True)


In [260]:
amex_all['no_of_children_1']=np.where(amex_all['no_of_children']=='1',1,0)
amex_all['no_of_children_2']=np.where(amex_all['no_of_children']=='2',1,0)
amex_all['no_of_children_3']=np.where(amex_all['no_of_children']=='3+',1,0)
amex_all.drop('no_of_children',axis=1,inplace=True)


In [261]:
train_all['age_range'].value_counts(dropna=False)

NaN      34708
46-55    15154
36-45    11210
26-35     8024
56-70     3723
70+       3094
18-25     2456
Name: age_range, dtype: int64

In [262]:
t=amex_all['age_range'].str.split('-',expand=True)

In [263]:
# t['age']=np.where(t[0]=='70+',70,np.where(t[0].isnull(),np.nan,0.5(t[0].astype(int)+t[1].astype(int))))
t[0]=pd.to_numeric(t[0],errors='coerce')
t[1]=pd.to_numeric(t[1],errors='coerce')

In [264]:
amex_all['age']=np.where(amex_all['age_range']=='70+',70,np.where(t[0].isnull(),np.nan,0.5*(t[0]+t[1])))

In [265]:
del amex_all['age_range']

In [68]:
amex_all.head()

,id,campaign_id,coupon_id,customer_id,redemption_status,duration,start_date_dow,start_date_day,end_date_dow,end_date_day,...,marital_status_single,family_size_1,family_size_2,family_size_3,family_size_4,family_size_5,no_of_children_1,no_of_children_2,no_of_children_3,age
0,1,13,27,1053,0.0,-12,6,139,1,127,...,0,1,0,0,0,0,0,0,0,50.5
1,2,13,116,48,0.0,-12,6,139,1,127,...,0,0,1,0,0,0,0,0,0,40.5
2,6,9,635,205,0.0,31,6,307,2,338,...,0,0,1,0,0,0,0,0,0,50.5
3,7,13,644,1050,0.0,-12,6,139,1,127,...,0,0,0,0,0,0,0,0,0,NaN
4,9,8,1017,1489,0.0,77,5,47,5,124,...,0,0,1,0,0,0,0,0,0,50.5


# Model building part

In [266]:
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

In [268]:
amex_all.drop(['campaign_id','coupon_id','customer_id'],axis=1,inplace=True)

In [271]:
amex_all.isnull().sum()

id                                      0
redemption_status                   50226
duration                                0
start_date_dow                          0
start_date_day                          0
end_date_dow                            0
end_date_day                            0
rented                                  0
income_bracket                          0
coupon_occurence_count                  0
coupon_unique_items_count               0
Bakery_cat_count                        0
Dairy, Juices & Snacks_cat_count        0
Flowers & Plants_cat_count              0
Garden_cat_count                        0
Grocery_cat_count                       0
Meat_cat_count                          0
Miscellaneous_cat_count                 0
Natural Products_cat_count              0
Packaged Meat_cat_count                 0
Pharmaceutical_cat_count                0
Prepared Food_cat_count                 0
Restauarant_cat_count                   0
Salads_cat_count                  

In [270]:
for col in amex_all.columns:
    if (col not in ['redemption_status','data'])& (amex_all[col].isnull().sum()>0):
        amex_all.loc[amex_all[col].isnull(),col]=amex_all.loc[amex_all['data']=='train',col].mean()

In [272]:
train_all=amex_all[amex_all['data']=='train']
del train_all['data']
test_all=amex_all[amex_all['data']=='test']
test_all.drop(['redemption_status','data'],axis=1,inplace=True)

C:\Users\deepak kumar\Anaconda3\lib\site-packages\pandas\core\frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [273]:
del amex_all

In [274]:
train1,train2=train_test_split(train_all,test_size=0.2,random_state=2)

In [275]:
x_train=train1.drop(['redemption_status'],axis=1)
y_train=train1['redemption_status']
x_test=train2.drop(['redemption_status'],axis=1)
y_test=train2['redemption_status']

In [276]:
x_train.reset_index(drop=True,inplace=True)
y_train.reset_index(drop=True,inplace=True)

In [134]:
def report(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.5f})".format(
                  results['mean_test_score'][candidate],
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")

In [277]:
x_train1=train_all.drop(['redemption_status'],axis=1)
y_train1=train_all['redemption_status']

# Using Xgboost

In [278]:
param_dist = {
              "max_depth": [2,3,4,5,6],
              "learning_rate":[0.01,0.05,0.1,0.3,0.5],
    "min_child_weight":[4,5,6],
              "subsample":[i/10.0 for i in range(6,10)],
 "colsample_bytree":[i/10.0 for i in range(6,10)],
               "reg_alpha":[1e-5, 1e-2, 0.1, 1, 100],
              "gamma":[i/10.0 for i in range(0,5)],
    "n_estimators":[100,500,700,1000],
    'scale_pos_weight':[2,3,4,5,6,7,8,9]
    
              }


In [279]:
from xgboost.sklearn import XGBClassifier

In [280]:
clf=XGBClassifier(objective='binary:logistic')

In [281]:
n_iter=10

random_search=RandomizedSearchCV(clf,n_jobs=-1,verbose=2,cv=10,n_iter=n_iter,scoring='roc_auc',
                                 param_distributions=param_dist)

In [ ]:
random_search.fit(x_train1,y_train1)


Fitting 10 folds for each of 10 candidates, totalling 100 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


In [169]:
report(random_search.cv_results_,5)

Model with rank: 1
Mean validation score: 0.930 (std: 0.00861)
Parameters: {'subsample': 0.7, 'scale_pos_weight': 2, 'reg_alpha': 1e-05, 'n_estimators': 500, 'min_child_weight': 5, 'max_depth': 6, 'learning_rate': 0.05, 'gamma': 0.4, 'colsample_bytree': 0.8}

Model with rank: 2
Mean validation score: 0.928 (std: 0.01334)
Parameters: {'subsample': 0.6, 'scale_pos_weight': 8, 'reg_alpha': 100, 'n_estimators': 500, 'min_child_weight': 6, 'max_depth': 4, 'learning_rate': 0.5, 'gamma': 0.0, 'colsample_bytree': 0.6}

Model with rank: 3
Mean validation score: 0.927 (std: 0.00867)
Parameters: {'subsample': 0.8, 'scale_pos_weight': 5, 'reg_alpha': 1, 'n_estimators': 500, 'min_child_weight': 5, 'max_depth': 3, 'learning_rate': 0.1, 'gamma': 0.0, 'colsample_bytree': 0.9}

Model with rank: 4
Mean validation score: 0.920 (std: 0.01831)
Parameters: {'subsample': 0.6, 'scale_pos_weight': 5, 'reg_alpha': 100, 'n_estimators': 700, 'min_child_weight': 5, 'max_depth': 5, 'learning_rate': 0.05, 'gamma': 0

In [191]:
xgb_best=XGBClassifier(subsample=0.95,scale_pos_weight=2,reg_alpha=1e-05,n_estimators=970,min_child_weight=6,
                       max_depth=12,learning_rate=0.35,gamma=0.95,colsample_bytree=0.8
                      )

In [192]:
xgb_best.fit(x_train1,y_train1)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=0.8, gamma=0.95,
       learning_rate=0.35, max_delta_step=0, max_depth=12,
       min_child_weight=6, missing=None, n_estimators=970, n_jobs=1,
       nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=1e-05, reg_lambda=1, scale_pos_weight=2, seed=None,
       silent=None, subsample=0.95, verbosity=1)

In [193]:
test_score=xgb_best.predict_proba(test_all)[:,1]

In [194]:
test_score

array([1.52213246e-01, 1.08303015e-04, 4.29212674e-02, ...,
       1.19240135e-02, 5.53393643e-03, 5.00383503e-05], dtype=float32)

In [90]:
pd.DataFrame(test_score).to_csv("Deepak_kumar.csv",index=False)

In [197]:
cutoffs=np.linspace(0.01,0.99,99)

cutoffs

array([0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1 , 0.11,
       0.12, 0.13, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19, 0.2 , 0.21, 0.22,
       0.23, 0.24, 0.25, 0.26, 0.27, 0.28, 0.29, 0.3 , 0.31, 0.32, 0.33,
       0.34, 0.35, 0.36, 0.37, 0.38, 0.39, 0.4 , 0.41, 0.42, 0.43, 0.44,
       0.45, 0.46, 0.47, 0.48, 0.49, 0.5 , 0.51, 0.52, 0.53, 0.54, 0.55,
       0.56, 0.57, 0.58, 0.59, 0.6 , 0.61, 0.62, 0.63, 0.64, 0.65, 0.66,
       0.67, 0.68, 0.69, 0.7 , 0.71, 0.72, 0.73, 0.74, 0.75, 0.76, 0.77,
       0.78, 0.79, 0.8 , 0.81, 0.82, 0.83, 0.84, 0.85, 0.86, 0.87, 0.88,
       0.89, 0.9 , 0.91, 0.92, 0.93, 0.94, 0.95, 0.96, 0.97, 0.98, 0.99])

In [198]:
train_score=xgb_best.predict_proba(x_train1)[:,1]

real=y_train1

In [199]:
     KS_cut=[]
     for cutoff in cutoffs:
        predicted=(train_score>cutoff).astype(int)
        
        TP=((predicted==1) & (real==1)).sum()
        TN=((predicted==0) & (real==0)).sum()
        FP=((predicted==1) & (real==0)).sum()
        FN=((predicted==0) & (real==1)).sum()
        
        P=TP+FN
        N=TN+FP
        KS=(TP/P)-(FP/N)
        KS_cut.append(KS)
        
        cutoff_data=pd.DataFrame(list(zip(cutoffs,KS_cut)),columns=["cutoff","KS"])
        
        KS_cutoff=cutoff_data[cutoff_data["KS"]==cutoff_data["KS"].max()]["cutoff"]
        

In [200]:
mycutoff=cutoffs[KS_cut==max(KS_cut)][0]  
mycutoff 

0.41000000000000003

In [201]:
test_score=xgb_best.predict_proba(test_all)[:,1]
test_score

array([1.52213246e-01, 1.08303015e-04, 4.29212674e-02, ...,
       1.19240135e-02, 5.53393643e-03, 5.00383503e-05], dtype=float32)

In [202]:
test_classes=(test_score>mycutoff).astype(int) 

In [203]:
pd.DataFrame(list(zip(test_all['id'],test_classes)),columns=["id","redemption_status"]).to_csv('xgboost_new.csv',index=False)


In [204]:
pd.DataFrame(list(zip(test_all['id'],test_score)),columns=["id","redemption_status"]).to_csv('xgboost1_new.csv',index=False)
